In [4]:
import numpy as np
import pandas as pd

In [17]:
base = pd.read_csv(filepath_or_buffer='BDSCLEANED.txt', sep = '\t')
base = base.drop('Unnamed: 0', axis = 1)
base

,Subject,Age,AgeGroup,Gender,Height,Weight,BMI,FootLen,Nationality,SkinColor,...,y_Mean_Resultant_Speed_cop - Closed - Foam,y_fpcntile50_cop - Closed - Foam,y_fpcntile95_cop - Closed - Foam,y_Peaky_Frequency_cop - Closed - Foam,y_Mean_Frequency_cop - Closed - Foam,y_Total_power_cop - Closed - Foam,APA,RPC,SO,DG
0,1,33.000000,Young,0,157.5,54.20,21.849332,21.80,Brazil,Yellow,...,0.026691,0.177778,0.844444,0.033333,0.255995,0.154036,6.0,4.0,6.0,9.0
1,2,27.583333,Young,0,154.0,44.00,18.552876,22.45,Brazil,Pardo/Brown,...,0.037090,0.255556,0.511111,0.266667,0.264554,0.418301,6.0,4.0,6.0,9.0
2,3,36.916667,Young,1,161.7,63.40,24.247626,23.10,Brazil,White,...,0.047679,0.222222,0.766667,0.111111,0.280833,0.512078,6.0,6.0,6.0,9.0
3,4,61.750000,Old,1,164.0,68.35,25.412701,24.70,Brazil,Pardo/Brown,...,0.066223,0.166667,0.577778,0.066667,0.214622,1.880497,6.0,6.0,6.0,8.0
4,5,18.333333,Young,0,162.0,56.45,21.509678,NaN,Brazil,White,...,0.046808,0.233333,0.966667,0.188889,0.318833,0.413409,6.0,6.0,6.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,159,24.750000,Young,0,153.5,62.05,26.334497,21.65,Brazil,White,...,0.038548,0.311111,0.988889,0.222222,0.382401,0.201918,6.0,6.0,6.0,10.0
154,160,23.583333,Young,0,161.5,56.55,21.681412,22.75,Brazil,White,...,0.052360,0.188889,0.688889,0.033333,0.243581,0.834123,5.0,4.0,6.0,9.0
155,161,25.333333,Young,0,167.7,60.55,21.530198,23.75,Brazil,White,...,0.068107,0.377778,1.177778,0.211111,0.421260,0.516105,6.0,4.0,6.0,9.0
156,162,24.083333,Young,1,183.5,63.20,18.769165,25.75,Brazil,White,...,0.055816,0.188889,1.066667,0.066667,0.326689,0.423302,6.0,5.0,6.0,9.0


In [15]:
subtestes={
    'APA': 0,
    'RPC': 0,
    'SO': 0,
    'DG': 0
}

In [34]:
x = base.drop([ 'Subject', 'AgeGroup','BMI',
       'FootLen', 'Nationality', 'SkinColor',
       'Ystudy', 'Footwear', 'Illness', 'Illness2', 'Nmedication',
       'Medication', 'Ortho-Prosthesis', 'Ortho-Prosthesis2', 'Disability',
       'Disability2', 'Falls12m', 'FES_1', 'FES_2', 'FES_3', 'FES_4', 'FES_5',
       'FES_6', 'FES_7', 'FES_T', 'FES_S', 'IPAQ_1a', 'IPAQ_1b', 'IPAQ_2a',
       'IPAQ_2b', 'IPAQ_3a', 'IPAQ_3b', 'IPAQ_4a', 'IPAQ_4b', 'IPAQ_S',
       'TMT_timeA', 'TMT_errorsA', 'TMT_timeB', 'TMT_errorsB', 'Best_1',
       'Best_2', 'Best_3l', 'Best_3r', 'Best_4', 'Best_5', 'Best_6l',
       'Best_6r', 'Best_7', 'Best_8', 'Best_9', 'Best_10', 'Best_11',
       'Best_12', 'Best_13', 'Best_14', 'Best_T','APA','RPC','SO', 'DG'], axis = 1)

#FEATURES DO RESNET
resnetfeatures = pd.read_csv(filepath_or_buffer='/Users/Nathan/Desktop/Machine Learning/Projeto de Extensão/2020/Outubro/heat_map_encoded.txt', sep = '\t')
resnetfeatures = resnetfeatures.reindex()
resnetfeatures.drop('Unnamed: 0',inplace = True, axis = 1)

#UNINDO AS FEATURES
x = x.join(resnetfeatures)

#Normaizando
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

x = pd.DataFrame(x)

In [35]:
#x=x.values
from sklearn.model_selection import KFold
#faz a separação 90 / 10
kf= KFold(n_splits=10)
kf.get_n_splits(x)

10

In [36]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

In [37]:
for subteste in subtestes:
    
    print('################################################')
    print('SUBTESTE',subteste)
    print('################################################')
    
    y= base[subteste]
    
    todos_mae=[]
    
    for train_index, test_index in kf.split(x):
        x_train, x_test = x.values[train_index], x.values[test_index] #gera os indices a serem treinados
        y_train, y_test = y[train_index], y[test_index] 

        #print('TREINAMENTO ATUAL:')
        #print("TRAIN:", train_index, "TEST:", test_index)

        regr=RandomForestRegressor(criterion='mse')
        regr.fit(x_train,y_train)

        previsoes= regr.predict(x_test)
        erro_médio = mean_absolute_error(y_test, previsoes)

        #print('Absolute Error:', erro_médio)
        
        todos_mae.append(erro_médio)

    mae_final=0

    for n in todos_mae:
        mae_final+= (n/ len(todos_mae))
            
    subtestes[subteste]= mae_final

################################################
SUBTESTE APA
################################################
################################################
SUBTESTE RPC
################################################
################################################
SUBTESTE SO
################################################
################################################
SUBTESTE DG
################################################


In [38]:
subtestes

{'APA': 0.7587291666666667,
 'RPC': 0.9754291666666669,
 'SO': 0.6770208333333334,
 'DG': 1.2069083333333332}

In [8]:
subtestes

{'APA': 0.7262166666666666,
 'RPC': 0.9441583333333333,
 'SO': 0.6347875000000001,
 'DG': 1.1704333333333334}

# RF PARA A PRIMEIRA PARTE DA MEDIANA

In [39]:
for i, subteste in enumerate(subtestes):
    
    #usando a mediana do BesTest
    
    X = x.copy()
    y = base.copy()
    mediana = np.median(base['Best_T'].values)
    
    # Caso queira usar a mediana do sub teste
    
    #mediana = np.median(base[subteste].values)
    #y = base[subteste].copy()

    
    print('################################################')
    print('SUBTESTE: ',subteste)
    #print('Mediana',mediana )
    print('Mediana do Best_T', mediana)
    print('################################################')
    
    
    #Criando a base de dados abaixo da mediana
    #Se for usar a mediana do subteste
    
    #for n in range(len(base.index)):
    #    if y[n] >=mediana:
    #        y.drop(labels=n, inplace = True)
    #       X.drop(labels=n, inplace = True)
    
    #Criando a base de dados abaixo da mediana
    #Usando a Mediana do Best_T
    delete= []
    for n in range(len(base.index)):
        if y.iloc[n,59] >= mediana:
            delete.append(n)
            
            
    y.drop(labels=delete, inplace = True)
    X.drop(labels=delete, inplace = True)
            
    #É necessário resetar o indice, pois o KFOLD precisa dos indices do row sequencialmente dispostos
    #Caso use a mediana dos subtestes comente a linha subsequente
    
    y = y[subteste]
    y = y.reset_index()
    y.drop('index', inplace= True, axis=1)
    y = y.values.ravel()
    
    
    X.reset_index(inplace= True)
    X.drop('index', inplace= True, axis=1)
    
    print('População ', len(X.index), ' De 158') 
    
    #Refazendo o Kfold
    
    kf= KFold(n_splits=10)
    kf.get_n_splits(X)
    
    todos_mae=[]
    
    for train_index, test_index in kf.split(X):
        x_train, x_test = X.values[train_index], X.values[test_index] #gera os indices a serem treinados
        y_train, y_test = y[train_index], y[test_index] 

        #print('TREINAMENTO ATUAL:')
        #print("TRAIN:", train_index, "TEST:", test_index)

        regr=RandomForestRegressor(criterion='mse')
        regr.fit(x_train,y_train)

        previsoes= regr.predict(x_test)
        erro_médio = mean_absolute_error(y_test, previsoes)

        #print('Absolute Error:', erro_médio)
        if subteste in ['APA','SO']:
            print('Previsão',previsoes )
            print('Teste',y_test )
        todos_mae.append(erro_médio)

        
        todos_mae.append(erro_médio)

    mae_final=0
    #print(todos_mae)
    for n in todos_mae:
        mae_final+= (n/ len(todos_mae))
    
            
    subtestes[subteste]= mae_final

################################################
SUBTESTE:  APA
Mediana do Best_T 22.0
################################################
População  75  De 158
Previsão [4.06 3.38 2.94 3.98 3.44 3.65 3.34 3.48]
Teste [5. 5. 4. 4. 4. 3. 4. 3.]
Previsão [3.22 3.52 3.11 3.82 3.8  3.73 3.51 3.13]
Teste [3. 4. 2. 4. 4. 5. 5. 4.]
Previsão [3.19 3.7  3.42 3.77 3.2  3.21 3.67 3.69]
Teste [2. 2. 2. 3. 2. 3. 3. 2.]
Previsão [3.46 3.63 3.1  3.69 3.8  2.48 2.83 3.4 ]
Teste [4. 2. 3. 4. 2. 3. 2. 4.]
Previsão [2.73 2.88 3.13 3.18 3.8  2.83 3.2  3.51]
Teste [4. 2. 3. 3. 3. 2. 3. 3.]
Previsão [3.44 3.53 3.62 3.97 3.37 3.2  3.92]
Teste [3. 3. 4. 4. 4. 3. 3.]
Previsão [3.08 3.54 3.19 3.21 3.37 3.55 3.29]
Teste [4. 4. 4. 3. 4. 4. 3.]
Previsão [2.45 4.16 3.37 3.05 2.95 3.37 3.58]
Teste [3. 5. 4. 2. 4. 1. 4.]
Previsão [3.44 3.91 3.88 3.13 2.75 3.42 3.17]
Teste [4. 4. 5. 5. 3. 3. 4.]
Previsão [3.58 3.3  2.97 3.14 3.35 3.74 3.25]
Teste [5. 4. 3. 2. 4. 3. 4.]
################################################
SUB

In [40]:
subtestes

{'APA': 0.7456607142857143,
 'RPC': 0.9235535714285714,
 'SO': 0.746482142857143,
 'DG': 0.9629821428571428}

#  RF ACIMA DA MEDIANA

In [41]:
for i, subteste in enumerate(subtestes):
    
    #usando a mediana do BesTest
    
    X = x.copy()
    y = base.copy()
    mediana = np.median(base['Best_T'].values)
    
    # Caso queira usar a mediana do sub teste
    
    #mediana = np.median(base[subteste].values)
    #y = base[subteste].copy()

    
    print('################################################')
    print('SUBTESTE: ',subteste)
    #print('Mediana',mediana )
    print('Mediana do Best_T', mediana)
    print('################################################')
    
    
    #Criando a base de dados abaixo da mediana
    #Se for usar a mediana do subteste
    
    #for n in range(len(base.index)):
    #    if y[n] >=mediana:
    #        y.drop(labels=n, inplace = True)
    #       X.drop(labels=n, inplace = True)
    
    #Criando a base de dados abaixo da mediana
    #Usando a Mediana do Best_T
    delete= []
    for n in range(len(base.index)):
        if y.iloc[n,59] < mediana:
            delete.append(n)
            
            
    y.drop(labels=delete, inplace = True)
    X.drop(labels=delete, inplace = True)
            
    #É necessário resetar o indice, pois o KFOLD precisa dos indices do row sequencialmente dispostos
    #Caso use a mediana dos subtestes comente a linha subsequente
    
    y = y[subteste]
    y = y.reset_index()
    y.drop('index', inplace= True, axis=1)
    y = y.values.ravel()
    
    
    X.reset_index(inplace= True)
    X.drop('index', inplace= True, axis=1)
    
    print('População ', len(X.index), ' De 158') 
    
    #Refazendo o Kfold
    
    kf= KFold(n_splits=10)
    kf.get_n_splits(X)
    
    todos_mae=[]
    
    for train_index, test_index in kf.split(X):
        x_train, x_test = X.values[train_index], X.values[test_index] #gera os indices a serem treinados
        y_train, y_test = y[train_index], y[test_index] 

        #print('TREINAMENTO ATUAL:')
        #print("TRAIN:", train_index, "TEST:", test_index)

        regr=RandomForestRegressor(criterion='mse')
        regr.fit(x_train,y_train)

        previsoes= regr.predict(x_test)
        erro_médio = mean_absolute_error(y_test, previsoes)

        #print('Absolute Error:', erro_médio)
        #if subteste in ['APA','SO']:
        #    print('Previsão',previsoes )
        #    print('Teste',y_test )
        #todos_mae.append(erro_médio)

        
        todos_mae.append(erro_médio)

    mae_final=0
    #print(todos_mae)
    for n in todos_mae:
        mae_final+= (n/ len(todos_mae))
    
            
    subtestes[subteste]= mae_final

################################################
SUBTESTE:  APA
Mediana do Best_T 22.0
################################################
População  83  De 158
################################################
SUBTESTE:  RPC
Mediana do Best_T 22.0
################################################
População  83  De 158
################################################
SUBTESTE:  SO
Mediana do Best_T 22.0
################################################
População  83  De 158
################################################
SUBTESTE:  DG
Mediana do Best_T 22.0
################################################
População  83  De 158


In [42]:
subtestes

{'APA': 0.5920277777777778,
 'RPC': 0.8639027777777777,
 'SO': 0.5574305555555555,
 'DG': 0.8943472222222223}